# IML Challenge ROOT TMVA Example

This is a binary classification example for the IML challenge. It implements a full toolchain for loading data files, train multiple classifiers and evaluate them. The used framework is [TMVA](https://root.cern.ch/root-user-guides-and-manuals) and optionally Keras with Theano.

## Set up TMVA

In [1]:
import ROOT

# Set up TMVA factory
output_file = ROOT.TFile.Open('TMVAClassification.root', 'RECREATE')
factory = ROOT.TMVA.Factory('TMVAClassification', output_file,
        '!V:!Silent:Color:!DrawProgressBar:Transformations=I,G:'+\
        'AnalysisType=Classification')

Welcome to JupyROOT 6.08/06


## Load data

**NOTE:** The data loaded here is already preprocessed. Checkout out the script `PREPROCESS_DATA` shipped with this notebook and run it in the terminal with ```sh PREPROCESS_DATA``` (you can do this on lxplus, after enabling the LCG software stack, or directly on a swan terminal).

Please note that the script only processes a small fraction of the events by default. Edit it if you want to process more. **IMPORTANT** do not use the full dataset if you run on swan, see the challenge README for details. 

The preprocessing does a zero-padding of the varying number of tracks and towers per event. We need to do this to include the given low-level information because we need a fix number of inputs for most MVA methods. For example, if we want to keep 5 tracks per event but we have only 3 in this event, then we are setting the track information of the 2 missing tracks to zero.

In [2]:
# NOTE: Check out the `preprocess_data.py` script shipped with this notebook!
data = ROOT.TFile("preprocessed_data.root")
if data == None:
    raise Exception('Have you run the preprocessing? Can not open file: {}'.format(filename))

quarks = data.Get('quarks')
gluons = data.Get('gluons')

# Set up dataloader and book all branches in the trees as input variables
dataloader = ROOT.TMVA.DataLoader('TMVAClassification')
for branch in quarks.GetListOfBranches():
    # NOTE: Here we are restricting the number of variables with rules on the variable name
    if not '_' in branch.GetName():
        dataloader.AddVariable(branch.GetName())

dataloader.AddTree(quarks, 'quarks', 1.0)
dataloader.AddTree(gluons, 'gluons', 1.0)
dataloader.PrepareTrainingAndTestTree(ROOT.TCut(''),
        # NOTE: Use this config if you run on the full dataset
        #'TrainTestSplit_quarks=0.8:TrainTestSplit_gluons=0.8:'+\
        'nTrain_quarks=9000:nTest_quarks=9000:'+\
        'nTrain_gluons=9000:nTest_gluons=9000:'+\
        'SplitMode=Random:NormMode=NumEvents:!V')

DataSetInfo              : [TMVAClassification] : Added class "quarks"
                         : Add Tree quarks of type quarks with 67450 events
DataSetInfo              : [TMVAClassification] : Added class "gluons"
                         : Add Tree gluons of type gluons with 59822 events
                         : Dataset[TMVAClassification] : Class index : 0  name : quarks
                         : Dataset[TMVAClassification] : Class index : 1  name : gluons


## Book MVA methods

In [3]:
# Fisher's discriminant
factory.BookMethod(dataloader, ROOT.TMVA.Types.kFisher, 'Fisher',
        'H:!V:VarTransform=None:Fisher')

# k-Nearest Neighbors
factory.BookMethod(dataloader, ROOT.TMVA.Types.kKNN, 'KNN',
        'H:!V:VarTransform=None:nkNN=5')

# Boosted Decision Tree
factory.BookMethod(dataloader, ROOT.TMVA.Types.kBDT, 'BDT',
        'H:!V:VarTransform=None:NTrees=100:MaxDepth=2:nCuts=10')

<ROOT.TMVA::MethodBDT object ("BDT") at 0x51ce790>

Factory                  : Booking method: Fisher
                         : 
Factory                  : Booking method: KNN
                         : 
Factory                  : Booking method: BDT
                         : 
DataSetFactory           : [TMVAClassification] : Number of events in input trees
                         : 
                         : 
                         : Number of training and testing events
                         : ---------------------------------------------------------------------------
                         : quarks -- training events            : 9000
                         : quarks -- testing events             : 9000
                         : quarks -- training and testing events: 18000
                         : gluons -- training events            : 9000
                         : gluons -- testing events             : 9000
                         : gluons -- training and testing events: 18000
                         : 
DataSetInf

**The following method PyKeras does only work with the software stack 88 or the bleeding edge software stack of SWAN!**

Please note, that running this notebook the first time can take some time because Theano compiles code in the background.

In [4]:
# Set Theano as backend of Keras
from os import environ
environ['KERAS_BACKEND'] = 'theano'

# Set architecture of system (AVX instruction set is not supported on SWAN and lxplus)
environ['THEANO_FLAGS'] = 'gcc.cxxflags=-march=corei7'

from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD

# Initialize PyMVA which implements the interface for Keras
ROOT.TMVA.PyMethodBase.PyInitialize()

# Set up neural network architecture
model = Sequential()
model.add(Dense(64, init='glorot_normal', activation='relu',
        input_dim=dataloader.GetDataSetInfo().GetNVariables()))
model.add(Dense(64, init='glorot_normal', activation='relu'))
model.add(Dense(2, init='glorot_uniform', activation='softmax'))

# Define loss, optimizer and validation metrics
model.compile(loss='categorical_crossentropy',
        optimizer=SGD(lr=1e-2),
        metrics=['accuracy',])

# Save model and print summary of set up architecture
filename_model = 'PyKerasModel.h5'
model.save(filename_model)
model.summary()

# Book method
factory.BookMethod(dataloader, ROOT.TMVA.Types.kPyKeras, 'PyKeras',
        'H:!V:VarTransform=G:FilenameModel={}:'.format(filename_model)+\
        'NumEpochs=5:BatchSize=100:SaveBestOnly=True:TriesEarlyStopping=1')

<ROOT.TMVA::MethodPyKeras object ("PyKeras") at 0x6802040>

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_1 (Dense)                  (None, 64)            448         dense_input_1[0][0]              
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 64)            4160        dense_1[0][0]                    
____________________________________________________________________________________________________
dense_3 (Dense)                  (None, 2)             130         dense_2[0][0]                    
Total params: 4738
____________________________________________________________________________________________________
Factory                  : Booking method: PyKeras
                         : 
PyKeras                  : [TMVAClassification] : Create Transformation "G" with events from a

Using Theano backend.
WARNING (theano.gof.cmodule): WARNING: your Theano flags `gcc.cxxflags` specify an `-march=X` flags.
         It is better to let Theano/g++ find it automatically, but we don't do it now
         It is better to let Theano/g++ find it automatically, but we don't do it now


## Train, test and evaluate methods

In [5]:
factory.TrainAllMethods()

Factory                  : Train all methods
Factory                  : [TMVAClassification] : Create Transformation "I" with events from all classes.
                         : 
                         : Transformation, Variable selection : 
                         : Input : variable 'jetPt' <---> Output : variable 'jetPt'
                         : Input : variable 'jetEta' <---> Output : variable 'jetEta'
                         : Input : variable 'jetPhi' <---> Output : variable 'jetPhi'
                         : Input : variable 'jetMass' <---> Output : variable 'jetMass'
                         : Input : variable 'ntracks' <---> Output : variable 'ntracks'
                         : Input : variable 'ntowers' <---> Output : variable 'ntowers'
Factory                  : [TMVAClassification] : Create Transformation "G" with events from all classes.
                         : 
                         : Transformation, Variable selection : 
                         : Input : va

In [6]:
factory.TestAllMethods()

Factory                  : Test all methods
Factory                  : Test method: Fisher for Classification performance
                         : 
Fisher                   : [TMVAClassification] : Evaluation of Fisher on testing sample (18000 events)
                         : Elapsed time for evaluation of 18000 events: 0.00806 sec       
Factory                  : Test method: KNN for Classification performance
                         : 
KNN                      : [TMVAClassification] : Evaluation of KNN on testing sample (18000 events)
                         : Elapsed time for evaluation of 18000 events: 1.86 sec       
Factory                  : Test method: BDT for Classification performance
                         : 
BDT                      : [TMVAClassification] : Evaluation of BDT on testing sample (18000 events)
                         : Elapsed time for evaluation of 18000 events: 0.105 sec       
Factory                  : Test method: PyKeras for Classification per

In [7]:
factory.EvaluateAllMethods()

Factory                  : Evaluate all methods
Factory                  : Evaluate classifier: Fisher
                         : 
Fisher                   : [TMVAClassification] : Loop over test events and fill histograms with classifier response...
                         : 
<WARNING>                : <Root> maximum iterations (100) reached before convergence
<WARNING>                : <Root> maximum iterations (100) reached before convergence
TFHandler_Fisher         : Variable        Mean        RMS   [        Min        Max ]
                         : -----------------------------------------------------------
                         :    jetPt:     112.20     8.4872   [     100.00     130.00 ]
                         :   jetEta:  0.0037413     1.4999   [    -4.4182     4.3669 ]
                         :   jetPhi:  0.0032062     1.8073   [    -3.1415     3.1413 ]
                         :  jetMass:     11.808     4.7094   [ 2.6974e-06     34.369 ]
                         : 

## Get results

In the following, we are plotting the receiver operating characteristic, which integral - the so called area under curve or ROC integral - is the figure of merit for this challenge. The explicit value is read out below and can be extracted from the TMVA evaluation output above as well.

As well, this example comes with a script `RUN_TMVA_GUI`, which can be executed from any terminal with connection to a graphical interface. It will run ROOT and open the full TMVA evaluation with all information taken from the file `TMVAClassification.root`, that is created by running this notebook.

In [8]:
# Plot ROC

# Enable JavaScript magic
%jsroot on

canvas = factory.GetROCCurve(dataloader)
canvas.Draw()

In [9]:
# Print area-under-curve (ROC integral) for Fisher method
print('AUC: {0}'.format(factory.GetROCIntegral(dataloader, 'Fisher')))

AUC: 0.773872494698


# Preparing the results submission

Please use the TMVAMeasureAUC.ipynb notebook to evaluate the AUC in the "modified" data set (for the results submission).